# Set Config and Complie

In [1]:
MODEL_DIR = './ML_CMSL1CaloTrigger/saved_models/qmodel_oct24'

In [2]:
import numpy as np
import h5py
import tensorflow as tf
import hls4ml

from qkeras import *
from qkeras.utils import load_qmodel

/afs/cern.ch/work/a/adpol/l1calotriggerad/synthesis/hls4ml/converters/__init__.py:24: UserWarning: WARNING: Pytorch converter is not enabled!
  warnings.warn("WARNING: Pytorch converter is not enabled!")


In [3]:
model = load_qmodel(MODEL_DIR);
model.summary()

2023-02-09 12:09:09.248034: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-02-09 12:09:09.248128: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (lxplus703.cern.ch): /proc/driver/nvidia/version does not exist
2023-02-09 12:09:09.412784: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.



Two checkpoint references resolved to different objects ({'class_name': 'quantized_bits', 'config': DictWrapper({'bits': 16, 'integer': 4, 'symmetric': True, 'alpha': 'auto', 'keep_negative': True, 'use_stochastic_rounding': False, 'qnoise_factor': 1.0}), 'shared_object_id': 2} and quantized_bits(16,4,1,alpha='auto')).



Two checkpoint references resolved to different objects ({'class_name': 'quantized_bits', 'config': DictWrapper({'bits': 16, 'integer': 4, 'symmetric': True, 'alpha': 'auto', 'keep_negative': True, 'use_stochastic_rounding': False, 'qnoise_factor': 1.0}), 'shared_object_id': 2} and quantized_bits(16,4,1,alpha='auto')).



Two checkpoint references resolved to different objects ({'class_name': 'quantized_bits', 'config': DictWrapper({'bits': 16, 'integer': 4, 'symmetric': True, 'alpha': 'auto', 'keep_negative': True, 'use_stochastic_rounding': False, 'qnoise_factor': 1.0}), 'shared_object_id': 9} and quantized_bits(16,4,1,alpha='auto')).



Two checkpoint references resolved to different objects ({'class_name': 'quantized_bits', 'config': DictWrapper({'bits': 16, 'integer': 4, 'symmetric': True, 'alpha': 'auto', 'keep_negative': True, 'use_stochastic_rounding': False, 'qnoise_factor': 1.0}), 'shared_object_id': 9} and quantized_bits(16,4,1,alpha='auto')).


...


Two checkpoint references resolved to different objects ({'class_name': 'quantized_bits', 'config': DictWrapper({'bits': 16, 'integer': 4, 'symmetric': True, 'alpha': 'auto', 'keep_negative': True, 'use_stochastic_rounding': False, 'qnoise_factor': 1.0}), 'shared_object_id': 9} and quantized_bits(16,4,1,alpha='auto')).



Two checkpoint references resolved to different objects ({'class_name': 'quantized_bits', 'config': DictWrapper({'bits': 16, 'integer': 4, 'symmetric': True, 'alpha': 'auto', 'keep_negative': True, 'use_stochastic_rounding': False, 'qnoise_factor': 1.0}), 'shared_object_id': 9} and quantized_bits(16,4,1,alpha='auto')).



Two checkpoint references resolved to different objects ({'class_name': 'quantized_bits', 'config': DictWrapper({'bits': 16, 'integer': 2, 'symmetric': True, 'alpha': 'auto', 'keep_negative': True, 'use_stochastic_rounding': False, 'qnoise_factor': 1.0}), 'shared_object_id': 15} and quantized_bits(16,2,1,alpha='auto')).



Two checkpoint references resolved to different objects ({'class_name': 'quantized_bits', 'config': DictWrapper({'bits': 16, 'integer': 2, 'symmetric': True, 'alpha': 'auto', 'keep_negative': True, 'use_stochastic_rounding': False, 'qnoise_factor': 1.0}), 'shared_object_id': 15} and quantized_bits(16,2,1,alpha='auto')).


Model: "model_35"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
In (InputLayer)              [(None, 252)]             0         
_________________________________________________________________
reshape (Reshape)            (None, 18, 14, 1)         0         
_________________________________________________________________
conv (QConv2D)               (None, 8, 6, 3)           27        
_________________________________________________________________
relu1 (QActivation)          (None, 8, 6, 3)           0         
_________________________________________________________________
flatten (Flatten)            (None, 144)               0         
_________________________________________________________________
dense1 (QDense)              (None, 20)                2880      
_________________________________________________________________
relu2 (QActivation)          (None, 20)                0  

In [4]:
hls4ml.model.optimizer.get_optimizer(
    'output_rounding_saturation_mode'
).configure(
    layers=['relu1', 'relu2'],
    rounding_mode='AP_RND',
    saturation_mode='AP_SAT',
    saturation_bits='AP_SAT'
)
hls4ml.model.optimizer.get_optimizer(
    'eliminate_linear_activation'
)

hls_config = hls4ml.utils.config_from_keras_model(
    model,
    granularity='name'
)

hls_config['Model']['ReuseFactor'] = 4
hls_config['Model']['Strategy'] = 'Resource'
hls_config['Model']['ClockPeriod']  = 6.25
hls_config['Model']['Precision'] = 'ap_fixed<16, 6>'
# hls_config['Model']['Trace']  = True

for layer in hls_config['LayerName'].keys():
    hls_config['LayerName'][layer]['Strategy'] = 'Resource'
    hls_config['LayerName'][layer]['ReuseFactor'] = 4
    # hls_config['LayerName'][layer]['Trace'] = True

hls_config['LayerName']['In']['Precision']['accum'] = 'ap_ufixed<10, 10>'
hls_config['LayerName']['In']['Precision']['result'] = 'ap_ufixed<10, 10>'

hls_config['LayerName']['conv']['Precision']['accum'] = 'ap_fixed<20, 8>'
hls_config['LayerName']['conv']['Precision']['result'] = 'ap_fixed<20, 8>'
hls_config['LayerName']['conv_linear']['Precision'] = 'ap_fixed<20, 8>'
hls_config['LayerName']['conv']['Strategy'] = 'Resource'
hls_config['LayerName']['conv']['ReuseFactor'] = 1
hls_config['LayerName']['conv']['ParallelizationFactor'] = 12

hls_config['LayerName']['dense1']['Precision']['accum'] = 'ap_fixed<22, 8>'
hls_config['LayerName']['dense1']['Precision']['result'] = 'ap_fixed<22, 8>'
hls_config['LayerName']['dense1_linear']['Precision'] = 'ap_fixed<22, 8>'
hls_config['LayerName']['dense1']['Strategy'] = 'Latency'

hls_config['LayerName']['output']['Precision']['accum'] = 'ap_fixed<16, 8>'
hls_config['LayerName']['output']['Precision']['result'] = 'ap_ufixed<16, 8>'
hls_config['LayerName']['output_linear']['Precision'] = 'ap_ufixed<16, 8>'

cfg = hls4ml.converters.create_config(part="xc7vx690tffg1927-2")

cfg['IOType'] = 'io_parallel'
cfg['HLSConfig'] = hls_config
cfg['KerasModel'] = model
cfg['ClockPeriod']  = 6.25
cfg['OutputDir']  = 'cicada/'
cfg['Part'] = 'xc7vx690tffg1927-2'

Interpreting Model
Topology:
Layer name: In, layer type: InputLayer, input shapes: [[None, 252]], output shape: [None, 252]
Layer name: reshape, layer type: Reshape, input shapes: [[None, 252]], output shape: [None, 18, 14, 1]
Layer name: conv, layer type: QConv2D, input shapes: [[None, 18, 14, 1]], output shape: [None, 8, 6, 3]
Layer name: relu1, layer type: Activation, input shapes: [[None, 8, 6, 3]], output shape: [None, 8, 6, 3]
Layer name: flatten, layer type: Reshape, input shapes: [[None, 8, 6, 3]], output shape: [None, 144]
Layer name: dense1, layer type: QDense, input shapes: [[None, 144]], output shape: [None, 20]
Layer name: relu2, layer type: Activation, input shapes: [[None, 20]], output shape: [None, 20]
Layer name: output, layer type: QDense, input shapes: [[None, 20]], output shape: [None, 1]


In [5]:
hls_model = hls4ml.converters.keras_to_hls(cfg)

hls4ml.model.optimizer.get_optimizer(
    'output_rounding_saturation_mode'
).configure(layers=[])
hls_model.compile()

Interpreting Model
Topology:
Layer name: In, layer type: InputLayer, input shapes: [[None, 252]], output shape: [None, 252]
Layer name: reshape, layer type: Reshape, input shapes: [[None, 252]], output shape: [None, 18, 14, 1]
Layer name: conv, layer type: QConv2D, input shapes: [[None, 18, 14, 1]], output shape: [None, 8, 6, 3]
Layer name: relu1, layer type: Activation, input shapes: [[None, 8, 6, 3]], output shape: [None, 8, 6, 3]
Layer name: flatten, layer type: Reshape, input shapes: [[None, 8, 6, 3]], output shape: [None, 144]
Layer name: dense1, layer type: QDense, input shapes: [[None, 144]], output shape: [None, 20]
Layer name: relu2, layer type: Activation, input shapes: [[None, 20]], output shape: [None, 20]
Layer name: output, layer type: QDense, input shapes: [[None, 20]], output shape: [None, 1]
Creating HLS model
Writing HLS project
Done


In [ ]:
hls4ml.utils.plot_model(hls_model, show_shapes=False, show_precision=True, to_file=None)

In [ ]:
hls4ml.model.profiling.numerical(model=model, hls_model=hls_model)

# Testing

In [ ]:
ACCEPTED_ERROR = 1.4

In [ ]:
# Test vector, zeros:
tv_0 = np.zeros((1, 252)) + 0.

# Test vector, ones:
tv_1 = np.zeros((1, 252)) + 1.

# Test vector, mean zero bias, 2018 run D:
tv_zb = np.array([[2, 1, 1, 2, 1, 2, 3, 3, 3, 2, 3, 1, 1, 2],
       [2, 1, 1, 2, 2, 2, 3, 3, 2, 2, 3, 1, 1, 2],
       [2, 1, 1, 1, 2, 2, 3, 3, 2, 2, 3, 1, 1, 2],
       [2, 1, 1, 2, 1, 2, 3, 3, 2, 1, 2, 1, 1, 2],
       [2, 1, 1, 2, 2, 2, 3, 4, 3, 2, 4, 1, 1, 2],
       [2, 1, 1, 2, 2, 2, 3, 3, 2, 2, 2, 1, 1, 2],
       [2, 1, 1, 3, 2, 3, 5, 2, 2, 1, 3, 1, 1, 2],
       [2, 1, 1, 2, 2, 3, 4, 3, 2, 2, 2, 1, 1, 3],
       [2, 1, 1, 2, 2, 3, 4, 3, 2, 1, 2, 1, 1, 3],
       [2, 1, 1, 2, 2, 2, 3, 3, 2, 1, 2, 1, 1, 2],
       [2, 1, 1, 2, 2, 2, 3, 3, 2, 1, 2, 1, 1, 2],
       [2, 1, 1, 2, 2, 2, 3, 3, 2, 1, 2, 1, 1, 2],
       [2, 1, 1, 2, 2, 2, 3, 3, 2, 1, 3, 1, 1, 2],
       [2, 1, 1, 2, 2, 2, 3, 3, 2, 1, 2, 1, 1, 2],
       [0, 0, 0, 2, 2, 2, 3, 3, 2, 2, 2, 1, 1, 2],
       [0, 0, 0, 3, 2, 3, 5, 4, 3, 2, 4, 1, 1, 2],
       [2, 1, 1, 2, 2, 2, 4, 3, 2, 2, 4, 1, 1, 2],
       [2, 1, 1, 2, 2, 2, 3, 3, 2, 1, 3, 1, 1, 2]]).reshape(1, 252) + 0.

# Test vector, signal haa4b_ma15_powheg average:
tv_sig = np.array([[8, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 8],
       [9, 2, 2, 3, 3, 3, 3, 3, 3, 2, 3, 2, 2, 9],
       [9, 2, 2, 3, 2, 3, 3, 3, 3, 3, 3, 2, 2, 9],
       [9, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 9],
       [8, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 8],
       [8, 2, 2, 3, 2, 3, 3, 3, 3, 3, 3, 2, 2, 8],
       [9, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 9],
       [9, 2, 2, 3, 3, 3, 3, 3, 3, 2, 3, 2, 2, 9],
       [9, 2, 2, 3, 3, 3, 3, 3, 3, 2, 3, 2, 2, 8],
       [8, 2, 2, 3, 3, 3, 3, 3, 3, 2, 3, 2, 2, 8],
       [9, 2, 2, 3, 3, 3, 3, 3, 3, 2, 3, 2, 2, 8],
       [9, 2, 2, 3, 2, 3, 3, 3, 3, 3, 3, 2, 2, 9],
       [9, 2, 2, 3, 2, 3, 3, 3, 3, 2, 3, 2, 2, 9],
       [8, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 8],
       [8, 2, 2, 3, 3, 3, 3, 3, 3, 2, 3, 2, 2, 8],
       [9, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 9],
       [9, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 9],
       [8, 2, 2, 3, 2, 3, 3, 3, 3, 2, 3, 2, 2, 8]]).reshape(1, 252) + 0.

# Test vector, hand made:
tv_hand = np.array([[2, 1, 1, 2, 1, 2, 3, 3, 3, 2, 3, 1, 1, 2],
       [2, 1, 1, 2, 2, 2, 3, 3, 2, 2, 3, 1, 1, 2],
       [2, 1, 1, 1, 2, 2, 1023, 3, 2, 2, 3, 1, 1, 2],
       [2, 1, 1, 2, 1, 2, 3, 3, 2, 1, 2, 1, 1, 2],
       [2, 1, 1, 1023, 1023, 2, 3, 4, 3, 2, 4, 1023, 1, 2],
       [2, 1, 1023, 1023, 2, 2, 3, 3, 2, 2, 2, 1, 1, 2],
       [2, 1, 1, 3, 2, 3, 5, 2, 2, 1, 3, 1023, 1, 2],
       [2, 1, 1, 2, 2, 3, 4, 3, 2, 2, 2, 1, 1, 3],
       [2, 1, 1, 2, 2, 3, 4, 3, 2, 1, 2, 1, 1, 3],
       [2, 1, 1, 2, 2, 2, 3, 3, 1023, 1, 2, 1, 1, 2],
       [2, 1, 1, 1023, 2, 2, 3, 3, 2, 1, 2, 1, 1, 2],
       [2, 1, 1, 2, 2, 2, 3, 3, 2, 1, 2, 1, 1, 2],
       [2, 1, 1, 2, 2, 2, 3, 3, 2, 1, 3, 1, 1, 2],
       [2, 1, 1, 2, 1023, 2, 3, 3, 2, 1, 2, 1, 1, 2],
       [0, 1023, 0, 2, 2, 2, 3, 3, 2, 2, 2, 1, 1, 2],
       [0, 0, 0, 3, 2, 3, 5, 4, 3, 2, 4, 1, 1, 2],
       [2, 1, 1, 2, 2, 2, 4, 3, 2, 2, 1023, 1, 1, 2],
       [2, 1023, 1, 2, 2, 2, 3, 3, 2, 1, 3, 1, 1, 2]]).reshape(1, 252) + 0.

In [ ]:
for tv in [tv_0, tv_1, tv_zb, tv_sig, tv_hand]:
    print(abs(model.predict(tv) - hls_model.predict(tv)) <= ACCEPTED_ERROR)

In [ ]:
# Real signal samples

scores = []
errors = []

for dataset in [
    '/eos/user/a/adpol/L1AD/Signal/120X/haa4taus_ma15_powheg.h5',
    '/eos/user/a/adpol/L1AD/Signal/120X/haa4b_ma50_powheg.h5',
    '/eos/user/a/adpol/L1AD/Signal/120X/emj-mMed-800-mDark-10-ctau-1000.h5']:
    X_test = h5py.File(dataset,'r')['CaloRegions']
    for idx in range(min(4000, len(X_test))):
        vector = X_test[idx].reshape(1, 252) + 0.
        score_keras = float(model.predict(vector))
        score_hls4ml = hls_model.predict(vector)
        diff = abs(score_keras - score_hls4ml)
        if diff > ACCEPTED_ERROR:
            print('Error', diff)
        errors.append(diff)
        scores.append(score_keras)

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
plt.scatter(scores, errors, s=1)
plt.xlabel('Anomaly Score, $S$', fontsize=18)
plt.ylabel('Error, $|S_{Keras} - S_{hls4ml}|$', fontsize=18)

# Debug

In [ ]:
model.predict(tv_0), hls_model.predict(tv_0)

In [ ]:
hls4ml_pred, hls4ml_trace = hls_model.trace(tv_0)
keras_trace = hls4ml.model.profiling.get_ymodel_keras(model, tv_0)

# Synthesis

In [ ]:
# Change the uncertainty in tcl: set_clock_uncertainty 30% {get_clocks default}

!grep -n create_clock cicada/build_prj.tcl | awk -F ":" '{print$1}' | ( read line; echo "$((line + 1)) ") | xargs -I {} sed -i '{} i set_clock_uncertainty 30% {get_clocks default}' cicada/build_prj.tcl 


In [ ]:
# Run Vivado HLS

!vivado_hls -f cicada/build_prj.tcl "reset=1 synth=1 csim=0 cosim=0 validation=0 export=0 vsynth=0"


In [ ]:
# Get the report

!cat cicada/myproject_prj/solution1/syn/report/myproject_csynth.rpt
